In [1]:
import utils 
from utils import *
from keras.callbacks import ReduceLROnPlateau


Using TensorFlow backend.


In [2]:
# plt.rcParams['figure.figsize'] = 10, 10
# %matplotlib inline
exp_name = "_5fold_fcn"

### Load Data

In [3]:
train = pd.read_json("data/input/train.json")
target_train=train['is_iceberg']
test = pd.read_json("data/input/test.json")


In [4]:
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
X_test_angle=test['inc_angle']

In [ ]:
# sample_weights = {0:}

### Preprocess

In [5]:


#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

### Train Model 

In [15]:


# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

In [34]:

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=6, min_lr=1e-6):
    es = EarlyStopping('val_loss', patience=patience)
    msave = ModelCheckpoint(filepath, save_best_only=True)
    lrplateu = ReduceLROnPlateau(monitor='val_loss',  patience=3, verbose=1, factor=0.5, min_lr=min_lr)

    return [es, msave,lrplateu]


In [30]:
getModel().summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 75, 75, 3)     0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 75, 75, 64)    1792        input_7[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Conv2D)            (None, 75, 75, 64)    36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 37, 37, 64)    0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [35]:
X_train[0].shape

(75, 75, 3)

In [38]:
dropout = 0.3
nf = 512
batch_size=128

def getModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    
    
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    for l in base_model.layers:
        l.trainable = False
        
    x = base_model.get_layer('block4_pool').output
    
    
    x = Convolution2D(nf,(3,3), activation='relu', padding='same', name="fcn_top")(x)
    x = BatchNormalization(axis=1)(x)    
    
    x = Convolution2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)    
    
    x = Convolution2D(nf,(3,3), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=1)(x)    
#     x = MaxPooling2D()(x)    
    
    x = GlobalMaxPooling2D()(x)
    
    
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(inputs=[base_model.input, input_2], outputs=predictions)
    
#     sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=1e-3),
                  metrics=['accuracy'])
    return model


#Using K-fold Cross Validation with Data Augmentation.
def trainKfold(X_train, X_angle, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=17).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    
    
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "weights/{}_{}.hdf5".format(exp_name,j+1)
        callbacks = get_callbacks(filepath=file_path, patience=10,min_lr=1e-6)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        
        model= getModel()
        model.fit_generator(
                gen_flow,
                steps_per_epoch=train.shape[0]/batch_size,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)
        
        
#         for l in model.layers:
#             l.trainable = True        
#         model.lr.set_value(1e-5)
#         callbacks = get_callbacks(filepath=file_path, patience=10,min_lr=1e-6)

#         model.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=train.shape[0]/batch_size,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)        
        
        
        #Getting the Best Model
        model.load_weights(filepath=file_path)
        #Getting Training Score
        score = model.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        
        #Getting Test Score
        score = model.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=model.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=model.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=model.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    
    train_log_loss = log_loss(target_train, y_train_pred_log) 
    valid_log_loss = log_loss(target_train, y_valid_pred_log)
    
    print('\n Train Log Loss Validation= ',train_log_loss)
    print(' Valid Log Loss Validation= ',valid_log_loss)
    
    
    return y_train_pred_log, y_valid_pred_log, y_test_pred_log, train_log_loss, valid_log_loss





In [39]:
train_preds , val_preds, test_preds, train_log_loss,valid_log_loss = trainKfold(X_train, X_angle, X_test)

('\n===================FOLD=', 1)
Epoch 1/100
12/12 [==============================] - 19s - loss: 1.7936 - acc: 0.5098 - val_loss: 7.5585 - val_acc: 0.5311
Epoch 2/100
12/12 [==============================] - 14s - loss: 0.8331 - acc: 0.4884 - val_loss: 7.5585 - val_acc: 0.5311
Epoch 3/100
12/12 [==============================] - 14s - loss: 0.7774 - acc: 0.4838 - val_loss: 7.5585 - val_acc: 0.5311
Epoch 4/100
12/12 [==============================] - 14s - loss: 0.7070 - acc: 0.5624 - val_loss: 6.7996 - val_acc: 0.5311
Epoch 5/100
12/12 [==============================] - 14s - loss: 0.6173 - acc: 0.6685 - val_loss: 1.0683 - val_acc: 0.7143
Epoch 6/100
12/12 [==============================] - 14s - loss: 0.4697 - acc: 0.7837 - val_loss: 1.2550 - val_acc: 0.6677
Epoch 7/100
12/12 [==============================] - 14s - loss: 0.4109 - acc: 0.7974 - val_loss: 0.5305 - val_acc: 0.8043
Epoch 8/100
12/12 [==============================] - 14s - loss: 0.3985 - acc: 0.8119 - val_loss: 0.6812 

12/12 [==============================] - 14s - loss: 0.2299 - acc: 0.9082 - val_loss: 0.3136 - val_acc: 0.8598
Epoch 57/100
11/12 [==========================>...] - ETA: 0s - loss: 0.2572 - acc: 0.8681
Epoch 00056: reducing learning rate to 7.81250037107e-06.
12/12 [==============================] - 14s - loss: 0.2555 - acc: 0.8720 - val_loss: 0.3116 - val_acc: 0.8598
Epoch 58/100
12/12 [==============================] - 13s - loss: 0.3075 - acc: 0.8753 - val_loss: 0.3097 - val_acc: 0.8598
('Train loss:', 0.2162031703922587)
('Train accuracy:', 0.89945440374123153)
('Test loss:', 0.30797351005478441)
('Test accuracy:', 0.85981308411214952)
('\n===================FOLD=', 3)
Epoch 1/100
12/12 [==============================] - 17s - loss: 1.1716 - acc: 0.5147 - val_loss: 7.5820 - val_acc: 0.5296
Epoch 2/100
12/12 [==============================] - 14s - loss: 0.7094 - acc: 0.5692 - val_loss: 7.5820 - val_acc: 0.5296
Epoch 3/100
12/12 [==============================] - 14s - loss: 0.6173 

11/12 [==========================>...] - ETA: 0s - loss: 0.3067 - acc: 0.8714
Epoch 00037: reducing learning rate to 6.25000029686e-05.
12/12 [==============================] - 14s - loss: 0.3003 - acc: 0.8732 - val_loss: 0.2299 - val_acc: 0.8969
Epoch 39/100
12/12 [==============================] - 14s - loss: 0.2409 - acc: 0.8863 - val_loss: 0.2258 - val_acc: 0.9000
('Train loss:', 0.22396710691422317)
('Train accuracy:', 0.8979750778816199)
('Test loss:', 0.21839119493961334)
('Test accuracy:', 0.89687499999999998)
('\n Train Log Loss Validation= ', 0.23703810137515618)
(' Valid Log Loss Validation= ', 0.28040256001874009)


In [41]:
pickle.dump(file=open("cache/{}_tmp_results.dmp".format(exp_name),"wb"), obj=[train_preds , val_preds, test_preds, train_log_loss,valid_log_loss])


In [17]:
train_preds , val_preds, test_preds, train_log_loss, valid_log_loss = pickle.load(file=open("cache/{}_tmp_results.dmp".format(exp_name),"rb"))


    

In [6]:
train_preds  = pd.DataFrame(data={"is_iceberg":train_preds})
train_preds["id"]=train['id'].astype(str)
train_preds.set_index("id",inplace=True)




In [7]:
val_preds  = pd.DataFrame(data={"id":train['id'],"is_iceberg":val_preds})
val_preds["id"]=train['id'].astype(str)
val_preds.set_index("id",inplace=True)

### Submission

In [8]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=test_preds
submission.to_csv('subm/{}.csv'.format(exp_name), index=False)

submission['id']=test['id'].astype(str)
submission.set_index("id",inplace=True)

In [23]:


def submit_and_shutdown(phase, exp_name, train_preds, val_preds,test_preds, train_log_loss,valid_log_loss,shutdown=True):

    lb  = 0.0
    try:
        submission_output = subprocess.check_output(["kg", "submit",'subm/{}.csv'.format(exp_name)])
        lb = float(submission_output)

    except Exception as e: 
        print (e)


    update_results_h5("ph1",exp_name,train_preds,val_preds,test_preds,train_log_loss,valid_log_loss,LB_score=lb)

    if shutdown:
        subprocess.call(["sudo", "shutdown", "now"])


def update_results_h5(phase, exp_name, train_id, test_id, train_preds, val_preds,test_preds, train_log_loss,valid_log_loss, LB_score=0.0 ):
    
    train_preds  = pd.DataFrame(data={"is_iceberg":train_preds})
    train_preds["id"]=train_id.astype(str)
    train_preds.set_index("id",inplace=True)

    
    
    val_preds  = pd.DataFrame(data={"id":train['id'],"is_iceberg":val_preds})
    val_preds["id"]=train_id.astype(str)
    val_preds.set_index("id",inplace=True)
    
    
    
    submission = pd.DataFrame()
    submission['id']=test_id
    submission['is_iceberg']=test_preds
    submission.to_csv('subm/{}.csv'.format(exp_name), index=False)

    submission['id']=test['id'].astype(str)
    submission.set_index("id",inplace=True)


    
    train_preds.to_hdf('data/results.h5',"/{}/train/{}".format(phase,exp_name))
    val_preds.to_hdf('data/results.h5',"/{}/valid/{}".format(phase,exp_name))
    submission.to_hdf('data/results.h5',"/{}/test/{}".format(phase,exp_name))

    
#     store = pd.HDFStore('data/results.h5')

#     store.append("/summary",pd.DataFrame(data={"phase":[phase],
#                                                "exp":[exp_name],
#                                                "train_log_loss":[train_log_loss],
#                                                "val_log_loss":[valid_log_loss], 
#                                                "LB":[LB_score] }) )  

#     store.close()

In [24]:
# submit_and_shutdown("ph1",exp_name,train_preds,val_preds,submission,train_log_loss,valid_log_loss,False)


In [20]:
train["id"].astype("str")

0       dfd5f913
1       e25388fd
2       58b2aaa0
3       4cfc3a18
4       271f93f4
5       b51d18b5
6       31da1a04
7       56929c16
8       525ab75c
9       192f56eb
10      3aac67cd
11      161a6860
12      3c794f0c
13      86730f0d
14      e356f7a3
15      87592c38
16      1c18a39e
17      a210f335
18      958d155f
19      6d81d201
20      75126706
21      112a6cfa
22      a29662a4
23      bd1a1bdf
24      31e37d93
25      76b8d446
26      958d42a8
27      70830858
28      faf2c49e
29      02314c59
          ...   
1574    84fe7f94
1575    04e6f331
1576    92c90853
1577    660a98a7
1578    89670962
1579    9d586019
1580    5f49ea3b
1581    968e1414
1582    389d7eaf
1583    65ca9e76
1584    a09cae27
1585    00c5b3e0
1586    7f9df2b0
1587    a2303efc
1588    cb62e5cb
1589    9ff1e0f0
1590    39fd995a
1591    544d0681
1592    cb0319fc
1593    d86deb2b
1594    cdee905a
1595    2539742b
1596    2ea3c9f1
1597    9cadda28
1598    8376a077
1599    04e11240
1600    c7d6f6f8
1601    bba1a0

In [25]:
update_results_h5("ph1",exp_name,train["id"], test["id"], 
                  train_preds,val_preds,test_preds,
                  train_log_loss,valid_log_loss,False)

In [26]:
store = pd.HDFStore('data/results.h5')


In [45]:
np.corrcoef(store.select("/ph1/valid/_5fold_baseline").values.ravel(), store.select("/ph1/train/_5fold_baseline").values.ravel())

array([[ 1.        ,  0.80484443],
       [ 0.80484443,  1.        ]])

In [ ]:
store.remove("")

In [27]:
store.keys()

['/summary',
 '/ph1/test/_5fold_baseline',
 '/ph1/test/_5fold_fcn',
 '/ph1/train/_5fold_baseline',
 '/ph1/train/_5fold_fcn',
 '/ph1/valid/_5fold_baseline',
 '/ph1/valid/_5fold_fcn']

In [39]:
store.get('/summary')

,LB,exp,phase,train_log_loss,val_log_loss
0,0.318,_5fold_baseline,ph1,0.305983,0.385856
0,0.000,_5fold_fcn,ph1,0.237038,0.280403
0,0.000,_5fold_fcn,ph1,0.237038,0.280403


In [82]:
benchmark3cv = pd.read_csv("subm/sub_benchmark.csv")

In [84]:
benchmark3cv.set_index('id',inplace=True)

In [92]:
benchmark3cv.join(store.get("/ph1/test/_5fold_baseline"),lsuffix="_3cv").corr()

,is_iceberg_3cv,is_iceberg
is_iceberg_3cv,1.000000,0.975071
is_iceberg,0.975071,1.000000
